In [1]:
import pandas as pd


In [ ]:
from pivottablejs import pivot_ui

In [ ]:
df = pd.read_csv('Sample.csv')
pivot_ui(df)

In [87]:
df = pd.read_csv('trans_six_months.csv')

In [4]:
df.head()

,account_id,date_trans,type,operation,amount,date_loan,status,days
0,10001,1996-01-14,credit,withdrawal_in_cash,500.0,1996-07-11,0.0,179
1,10001,1996-01-17,credit,withdrawal_in_cash,5700.0,1996-07-11,0.0,176
2,10001,1996-01-23,credit,withdrawal_in_cash,8500.0,1996-07-11,0.0,170
3,10001,1996-01-26,credit,withdrawal_in_cash,2500.0,1996-07-11,0.0,167
4,10001,1996-01-29,credit,withdrawal_in_cash,1900.0,1996-07-11,0.0,164


In [5]:
df['type'].value_counts()

credit        16360
withdrawal     9722
VYBER           920
Name: type, dtype: int64

In [6]:
df.dtypes

account_id      int64
date_trans     object
type           object
operation      object
amount        float64
date_loan      object
status        float64
days            int64
dtype: object

In [7]:
df.account_id = df.account_id.apply(lambda x: str(x))

In [8]:
df.dtypes

account_id     object
date_trans     object
type           object
operation      object
amount        float64
date_loan      object
status        float64
days            int64
dtype: object

In [28]:
a = pd.DataFrame(df.groupby(['account_id', 'type', 'operation'])['amount'].sum())

In [55]:
a_count = pd.DataFrame(df.groupby(['account_id', 'type', 'operation']).count())

In [59]:
a_count.columns

Index(['date_trans', 'amount', 'date_loan', 'status', 'days'], dtype='object')

In [64]:
a_count.reset_index(inplace = True)

In [69]:
a_count.operation.value_counts()

withdrawal_in_cash              1012
other                            679
credit_in_cash                   570
remittance_to_another_bank       407
collection_from_another_bank     210
credit_card_withdrawal            22
Name: operation, dtype: int64

In [107]:
df.columns

Index(['account_id', 'date_trans', 'type', 'operation', 'amount', 'date_loan',
       'status', 'days'],
      dtype='object')

In [117]:
df[df.operation == 'other'].head()

,account_id,date_trans,type,operation,amount,date_loan,status,days
5,10001,1996-01-31,withdrawal,other,189.3,1996-07-11,0.0,162
10,10001,1996-02-29,withdrawal,other,155.2,1996-07-11,0.0,133
15,10001,1996-03-31,withdrawal,other,187.6,1996-07-11,0.0,102
21,10001,1996-04-30,withdrawal,other,201.9,1996-07-11,0.0,72
27,10001,1996-05-31,withdrawal,other,186.5,1996-07-11,0.0,41


In [120]:
df['type'][df['operation']=='other'].nunique()

1

# Rename the operations

In [90]:
#### Aggregate on the basis of the operation vs ampunt
table_oper_amount = (pd.pivot_table(df, values='amount', index=['account_id'],columns=['operation'],
                         aggfunc=np.sum)).reset_index()

In [98]:
table_oper_amount[table_oper_amount.account_id == 10001]

operation,account_id,collection_from_another_bank,credit_card_withdrawal,credit_in_cash,other,remittance_to_another_bank,withdrawal_in_cash
592,10001,NaN,NaN,104546.0,1076.4,NaN,108587.6


In [101]:
### Drop the duplicate row on the basis of the status
table_status = df[['account_id', 'status']].drop_duplicates(subset='account_id', keep='last')

In [103]:
table_status.head()

,account_id,status
35,10001,0.0
68,10005,0.0
94,10019,0.0
127,10022,0.0
182,1004,0.0


In [104]:
### Merge the table table_status and table_oper_amount
trans_loan_final = pd.merge(table_oper_amount,table_status,on="account_id",how="left")

In [106]:
trans_loan_final.head()

operation,account_id,collection_from_another_bank,credit_card_withdrawal,credit_in_cash,other,remittance_to_another_bank,withdrawal_in_cash,status
0,2,131534.0,NaN,NaN,861.7,36330.0,113514.6,0.0
1,19,98403.0,NaN,17800.0,770.4,NaN,121373.0,1.0
2,25,NaN,NaN,250065.0,1585.7,41565.0,243365.6,0.0
3,37,NaN,NaN,151468.0,705.9,29988.0,134292.0,1.0
4,38,89967.0,NaN,8100.0,937.8,29671.0,78616.0,0.0


In [71]:
table_oper_amount.head()

operation,account_id,collection_from_another_bank,credit_card_withdrawal,credit_in_cash,other,remittance_to_another_bank,withdrawal_in_cash
0,10001,NaN,NaN,104546.0,1076.4,NaN,108587.6
1,10005,228313.0,NaN,NaN,1309.0,36604.0,210474.8
2,10019,NaN,NaN,76936.0,678.2,NaN,57687.6
3,10022,149908.0,NaN,NaN,1460.1,35346.0,118688.8
4,1004,NaN,NaN,410729.0,1385.1,NaN,437710.6


In [68]:
a_count.head()

,account_id,type,operation,date_trans,amount,date_loan,status,days
0,10001,credit,withdrawal_in_cash,24,24,24,24,24
1,10001,withdrawal,credit_in_cash,6,6,6,6,6
2,10001,withdrawal,other,6,6,6,6,6
3,10005,VYBER,withdrawal_in_cash,1,1,1,1,1
4,10005,credit,remittance_to_another_bank,4,4,4,4,4


In [29]:
type(a)

pandas.core.frame.DataFrame

In [34]:
a.reset_index()

,account_id,type,operation,amount
0,10001,credit,withdrawal_in_cash,108587.6
1,10001,withdrawal,credit_in_cash,104546.0
2,10001,withdrawal,other,1076.4
3,10005,VYBER,withdrawal_in_cash,10631.0
4,10005,credit,remittance_to_another_bank,36604.0
5,10005,credit,withdrawal_in_cash,199843.8
6,10005,withdrawal,collection_from_another_bank,228313.0
7,10005,withdrawal,other,1309.0
8,10019,credit,withdrawal_in_cash,57687.6
9,10019,withdrawal,credit_in_cash,76936.0


In [36]:
a.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2900 entries, (10001, credit, withdrawal_in_cash) to (993, withdrawal, other)
Data columns (total 1 columns):
amount    2900 non-null float64
dtypes: float64(1)
memory usage: 45.3+ KB


In [43]:
df.h

,account_id,date_trans,type,operation,amount,date_loan,status,days
0,10001,1996-01-14,credit,withdrawal_in_cash,500.0,1996-07-11,0.0,179
1,10001,1996-01-17,credit,withdrawal_in_cash,5700.0,1996-07-11,0.0,176
2,10001,1996-01-23,credit,withdrawal_in_cash,8500.0,1996-07-11,0.0,170
3,10001,1996-01-26,credit,withdrawal_in_cash,2500.0,1996-07-11,0.0,167
4,10001,1996-01-29,credit,withdrawal_in_cash,1900.0,1996-07-11,0.0,164


In [44]:
df.reset_index()

,index,account_id,date_trans,type,operation,amount,date_loan,status,days
0,0,10001,1996-01-14,credit,withdrawal_in_cash,500.0,1996-07-11,0.0,179
1,1,10001,1996-01-17,credit,withdrawal_in_cash,5700.0,1996-07-11,0.0,176
2,2,10001,1996-01-23,credit,withdrawal_in_cash,8500.0,1996-07-11,0.0,170
3,3,10001,1996-01-26,credit,withdrawal_in_cash,2500.0,1996-07-11,0.0,167
4,4,10001,1996-01-29,credit,withdrawal_in_cash,1900.0,1996-07-11,0.0,164
5,5,10001,1996-01-31,withdrawal,other,189.3,1996-07-11,0.0,162
6,6,10001,1996-01-31,credit,withdrawal_in_cash,14.6,1996-07-11,0.0,162
7,7,10001,1996-02-06,withdrawal,credit_in_cash,16084.0,1996-07-11,0.0,156
8,8,10001,1996-02-17,credit,withdrawal_in_cash,500.0,1996-07-11,0.0,145
9,9,10001,1996-02-22,credit,withdrawal_in_cash,14300.0,1996-07-11,0.0,140


In [45]:
df.columns

Index(['account_id', 'date_trans', 'type', 'operation', 'amount', 'date_loan',
       'status', 'days'],
      dtype='object')

In [46]:
b = df.pivot(index = 'account_id', columns='type',values='amount')

ValueError: Index contains duplicate entries, cannot reshape

In [15]:
df.dtypes

account_id     object
date_trans     object
type           object
operation      object
amount        float64
date_loan      object
status        float64
days            int64
dtype: object

In [17]:
import numpy as np

In [33]:
a.dtypes

amount    float64
dtype: object

In [18]:
table1 = (pd.pivot_table(df, values='amount', index=['account_id'],columns=['type'],
                         aggfunc=np.sum)).reset_index()

In [19]:
type(table1)

pandas.core.frame.DataFrame

In [21]:
table1.head()

type,account_id,VYBER,credit,withdrawal
0,10001,NaN,108587.6,105622.4
1,10005,10631.0,236447.8,229622.0
2,10019,NaN,57687.6,77614.2
3,10022,30245.0,123789.8,151368.1
4,1004,29723.0,407987.6,412114.1


In [31]:
table2 = (pd.pivot_table(df, index = 'account_id', columns= 'operation', values = 'amount',
                        aggfunc = np.sum)).reset_index()

In [32]:
table2

operation,account_id,collection_from_another_bank,credit_card_withdrawal,credit_in_cash,other,remittance_to_another_bank,withdrawal_in_cash
0,10001,NaN,NaN,104546.0,1076.4,NaN,108587.6
1,10005,228313.0,NaN,NaN,1309.0,36604.0,210474.8
2,10019,NaN,NaN,76936.0,678.2,NaN,57687.6
3,10022,149908.0,NaN,NaN,1460.1,35346.0,118688.8
4,1004,NaN,NaN,410729.0,1385.1,NaN,437710.6
5,10049,NaN,NaN,65677.0,436.4,17340.0,57207.6
6,10063,252863.0,NaN,4200.0,1287.5,47052.0,225290.0
7,10065,NaN,NaN,296720.0,1345.8,NaN,274467.6
8,10068,NaN,NaN,167511.0,931.2,NaN,110743.8
9,10079,NaN,NaN,254150.0,1456.3,NaN,263916.6
